In [1]:
import numpy as np
import cv2
import LOCOANSOverlay
CodecOverlay=LOCOANSOverlay.LOCOANSOverlay
import os

## Load overlay

In [2]:
codec = CodecOverlay("./platform.bit")

In [3]:
codec.print_codec_status()

idma  0:  Idle, 
idma  1:  Idle, 
odma  0:  Idle, 
odma  1:  Idle, 
decorrelator  0:  Idle, 
decorrelator  1:  Idle, 


## Set clocks
Adjust clk0 and clk1 according to the implementation results

In [4]:
set_clock=True
clk0_in_mhz = 80
clk1_in_mhz =  180

if set_clock:
    codec.clock0=clk0_in_mhz
    codec.clock1=clk1_in_mhz
    
print("Current. clock 0:",codec.clock0)
print("Current. clock 1:",codec.clock1)

Current. clock 0: 83.333333
Current. clock 1: 166.666667


/usr/local/lib/python3.6/dist-packages/pynq/ps.py:312: UserWarning: Setting frequency to the closet possible value 83.33333MHz.
  round(freq_high / q0, 5)))
/usr/local/lib/python3.6/dist-packages/pynq/ps.py:312: UserWarning: Setting frequency to the closet possible value 166.66667MHz.
  round(freq_high / q0, 5)))


## Run tests

### Test images

In [5]:
iters = 1
img_dir="./images/"
max_near= 4
max_near = max_near if max_near<= codec.max_near else codec.max_near
for file in os.listdir(img_dir):
    if not (file.endswith(".pgm")):
        continue
    image_name = str(file)
    image_path = img_dir+ image_name
    print("Image: ",file)
    
    for near in range(max_near+1):
        print("\tNear ",near," | ",end="")
        codec.near = near
        status,time_prof = codec.run_test(image_path,iters)
        correct, max_error = status
        assert correct,"Error running test using file %s with near %d: peak error: %d " % (
                image_name,near, max_error)

        img_pxs= codec.input_img.flatten().size
        bpp=codec.get_bpp() 
        print("Bw: %f Mpixels/s | bpp: %5.3f" % (img_pxs/ (time_prof["run_hw"]*10**6),bpp))
        


Image:  1000x1320_ales-krivec-434.pgm
	Near  0  | Bw: 81.688587 Mpixels/s | bpp: 4.293
	Near  1  | Bw: 80.823025 Mpixels/s | bpp: 2.801
	Near  2  | Bw: 81.747640 Mpixels/s | bpp: 2.183
	Near  3  | Bw: 81.758747 Mpixels/s | bpp: 1.815
	Near  4  | Bw: 81.745418 Mpixels/s | bpp: 1.557


### Constant images

In [6]:
iters = 5
codec.near = 0
img_dir="./images/"
for size in [10,100,1000,1500]: 
    print("Size ",size,"x",size)
    status,time_prof = codec.run_test(cols=size,rows=size,random_image=False,run_iterations=iters)
    correct, max_error = status
    assert correct,"Error running test using size %s with near %d: peak error: %d " % (
            image_nsizeame,near, max_error)
    
    img_pxs= codec.input_img.flatten().size
    bpp=codec.get_bpp() 
    print("\tBw: %f Mpixels/s | bpp: %5.3f" % (img_pxs/ (time_prof["run_hw"]*10**6),bpp))
    

Size  10 x 10
	Bw: 0.216640 Mpixels/s | bpp: 2.720
Size  100 x 100
	Bw: 23.622926 Mpixels/s | bpp: 0.067
Size  1000 x 1000
	Bw: 81.173600 Mpixels/s | bpp: 0.035
Size  1500 x 1500
	Bw: 82.358099 Mpixels/s | bpp: 0.035


### Random images

In [7]:
iters = 1
max_near=1
img_dir="./images/"
np.random.seed(0)
max_near = max_near if max_near<= codec.max_near else codec.max_near
for size in [10,100,1000]:
    print("Size: ",size)
    rows=size
    cols=size
    for near in range(max_near+1):
        print("\tNear ",near," | ",end="")
        codec.near = near
        status,time_prof = codec.run_test(cols=cols,rows=rows,random_image=True,run_iterations=iters)
        correct, max_error = status
        assert correct,"Error running test using size %s with near %d: peak error: %d " % (
                image_nsizeame,near, max_error)

        img_pxs= codec.input_img.flatten().size
        bpp=codec.get_bpp() 
        print("Bw: %f Mpixels/s | bpp: %5.3f" % (img_pxs/ (time_prof["run_hw"]*10**6),bpp))


Size:  10
	Near  0  | Bw: 0.226819 Mpixels/s | bpp: 18.480
	Near  1  | Bw: 0.240071 Mpixels/s | bpp: 13.440
Size:  100
	Near  0  | Bw: 16.425674 Mpixels/s | bpp: 9.235
	Near  1  | Bw: 20.164909 Mpixels/s | bpp: 7.324
Size:  1000
	Near  0  | Bw: 45.216485 Mpixels/s | bpp: 8.655
	Near  1  | Bw: 76.839169 Mpixels/s | bpp: 6.907
